In [1]:
import random as rn
import os
from itertools import product
from PIL import Image
import numpy as np
from plotly.figure_factory import create_distplot
import plotly.graph_objects as go

import tensorflow as tf
# from keras.models import load_model

from func import read_img, set_model, fit_model, evaluate_model, plot_history, create_data

# Ensure the environment is set to deterministic operations
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)


os.environ['PYTHONHASHSEED'] = '0'

# Disable parallelism to ensure operations are executed in a consistent order
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# Optionally disable GPU for full control (GPU computations can be non-deterministic)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

seed = 0
np.random.seed(seed)
rn.seed(seed)
tf.random.set_seed(seed)

In [2]:
# Count data

total_amount_train, total_amount_test = 0, 0
for dir in os.listdir('data/train'):

    print(dir, end=': ')
    count_train = len(os.listdir(f'data/train/{dir}'))
    count_test = len(os.listdir(f'data/test/{dir}'))
    print(f'{count_train} /', count_test)
    total_amount_train += count_train
    total_amount_test += count_test

print(f'TOTAL AMOUNT train, test: ', f'{total_amount_train} /', total_amount_test)

Avulsion fracture: 98 / 43
Comminuted fracture: 153 / 66
Compression-Crush fracture: 105 / 45
Fracture Dislocation: 111 / 48
Greenstick fracture: 95 / 41
Hairline Fracture: 97 / 42
Impacted fracture: 112 / 49
Intra-articular fracture: 72 / 32
Longitudinal fracture: 90 / 39
Oblique fracture: 86 / 38
Pathological fracture: 90 / 39
Spiral Fracture: 93 / 41
TOTAL AMOUNT train, test:  1202 / 523


# Data preparation

In [3]:
# Find max image shape in dataset

shape_width_max, shape_height_max = 0, 0
shape_lst = []
for dir in os.listdir('data/train'):
    for file in os.listdir(f'data/train/{dir}'):
        img = read_img(f'data/train/{dir}/{file}', as_array=True, black_white=True)

        shape_width, shape_height = img.shape

        shape_lst.append(img.shape)

shape_max = max([i[0] for i in shape_lst]), max([i[1] for i in shape_lst])
shape_max

c:\Users\Stas\anaconda3\envs\bone_frac_env\lib\site-packages\PIL\Image.py:1056: UserWarning:

Palette images with Transparency expressed in bytes should be converted to RGBA images



(6714, 4430)

In [4]:
# Image height and width distributions

fig = create_distplot(
    hist_data=[[i[0] for i in shape_lst], [i[1] for i in shape_lst]],
    group_labels=['width', 'height'],
    bin_size=100,
    histnorm='probability',
    show_hist=True,
    show_curve=False,
    show_rug=False,
)
fig

In [ ]:
# Fraction of images with shape less than pad_shape_value for each axis

pad_shape_value = 250
pad_shape = (pad_shape_value, pad_shape_value)
len([i for i in shape_lst if i[0] < pad_shape_value and i[1] < pad_shape_value]) / len(shape_lst)

0.24958402662229617

In [7]:
data_dir_name = 'data_processed'

In [ ]:
# Process images with format, not allowed by keras (turn it to .png)

allowed_formats = ['jpeg', 'png', 'bmp', 'gif']
allowed_formats += [i.upper() for i in allowed_formats]

for i, sample in enumerate(('train', 'test')):
    for j, dir in enumerate(os.listdir(f'{data_dir_name}/{sample}')):

        path = f'{data_dir_name}/{sample}/{dir}'
        # Read one label data
        for img_name in os.listdir(path):

            img_path = f'{path}/{img_name}'
            img = Image.open(img_path)

            if img.format not in allowed_formats:

                img.convert("RGB")

                name = img_name.rsplit('.', maxsplit=1)[0]
                img_name_new = f'{name}.png'
                img_path_new = f'{path}/{img_name_new}'

                img.save(img_path_new)
                img.close()
                os.remove(img_path)

# Model structure optimization

In [ ]:
train_data, valid_data, test_data = create_data(data_dir_name=data_dir_name, batch_size=50, pad_shape=pad_shape)

fig_loss = go.Figure(layout={'title': 'Loss'})
fig_accuracy = go.Figure(layout={'title': 'Accuracy'})

model = set_model(2.6e-3, 60, 0.4, pad_shape=pad_shape)

his = fit_model(model, train_data, valid_data, n_epochs=50)

fig_loss, fig_accuracy = plot_history(
    history=his,
    params=(2.6e-3, 60, 0.4),
    fig_loss=fig_loss,
    fig_accuracy=fig_accuracy,
)

metrics = evaluate_model(model, [train_data, valid_data, test_data])

Found 1202 files belonging to 12 classes.
Using 1022 files for training.
Found 1202 files belonging to 12 classes.
Using 180 files for validation.
Found 523 files belonging to 12 classes.
Epoch 1/50
21/21 [==============================] - 632s 30s/step - loss: 2.5635 - accuracy: 0.1008 - val_loss: 2.4818 - val_accuracy: 0.0889
Epoch 2/50
21/21 [==============================] - 699s 34s/step - loss: 2.4845 - accuracy: 0.1086 - val_loss: 2.4800 - val_accuracy: 0.0722
Epoch 3/50
21/21 [==============================] - 464s 22s/step - loss: 2.4802 - accuracy: 0.0959 - val_loss: 2.4809 - val_accuracy: 0.1167
Epoch 4/50
21/21 [==============================] - 385s 18s/step - loss: 2.4671 - accuracy: 0.1076 - val_loss: 2.4786 - val_accuracy: 0.1333
Epoch 5/50
21/21 [==============================] - 338s 16s/step - loss: 2.4543 - accuracy: 0.1233 - val_loss: 2.4751 - val_accuracy: 0.1000
Epoch 6/50
21/21 [==============================] - 307s 14s/step - loss: 2.4389 - accuracy: 0.1429 - 

In [23]:
# Add dropout to every convolution
fig_accuracy

In [ ]:
train_data, valid_data, test_data = create_data(data_dir_name=data_dir_name, batch_size=50, pad_shape=pad_shape)

fig_loss = go.Figure(layout={'title': 'Loss'})
fig_accuracy = go.Figure(layout={'title': 'Accuracy'})

model = set_model(2.6e-3, 60, 0.4, pad_shape=pad_shape)

his = fit_model(model, train_data, valid_data, n_epochs=50)

fig_loss, fig_accuracy = plot_history(
    history=his,
    params=(2.6e-3, 60, 0.4),
    fig_loss=fig_loss,
    fig_accuracy=fig_accuracy,
)

metrics = evaluate_model(model, [train_data, valid_data, test_data])

Found 1202 files belonging to 12 classes.
Using 1022 files for training.
Found 1202 files belonging to 12 classes.
Using 180 files for validation.
Found 523 files belonging to 12 classes.
Epoch 1/50


21/21 [==============================] - 86s 4s/step - loss: 2.4863 - accuracy: 0.0744 - val_loss: 2.4816 - val_accuracy: 0.0722
Epoch 2/50
21/21 [==============================] - 86s 4s/step - loss: 2.4794 - accuracy: 0.0910 - val_loss: 2.4730 - val_accuracy: 0.1333
Epoch 3/50
21/21 [==============================] - 78s 4s/step - loss: 2.4774 - accuracy: 0.1213 - val_loss: 2.4754 - val_accuracy: 0.1333
Epoch 4/50
21/21 [==============================] - 75s 3s/step - loss: 2.4720 - accuracy: 0.1125 - val_loss: 2.4694 - val_accuracy: 0.1333
Epoch 5/50
21/21 [==============================] - 76s 4s/step - loss: 2.4654 - accuracy: 0.1194 - val_loss: 2.4688 - val_accuracy: 0.1333
Epoch 6/50
21/21 [==============================] - 84s 4s/step - loss: 2.4654 - accuracy: 0.1076 - val_loss: 

In [19]:
# Add another hidden layer to fully connected part with dropout
fig_accuracy

In [ ]:
train_data, valid_data, test_data = create_data(data_dir_name=data_dir_name, batch_size=50, pad_shape=pad_shape)

fig_loss = go.Figure(layout={'title': 'Loss'})
fig_accuracy = go.Figure(layout={'title': 'Accuracy'})

model = set_model(5e-3, 60, 0.4, pad_shape=pad_shape)

his = fit_model(model, train_data, valid_data, n_epochs=50)

fig_loss, fig_accuracy = plot_history(
    history=his,
    params=(2.6e-3, 60, 0.4),
    fig_loss=fig_loss,
    fig_accuracy=fig_accuracy,
)

metrics = evaluate_model(model, [train_data, valid_data, test_data])

Found 1202 files belonging to 12 classes.
Using 1022 files for training.
Found 1202 files belonging to 12 classes.
Using 180 files for validation.
Found 523 files belonging to 12 classes.


Epoch 1/50


21/21 [==============================] - 92s 4s/step - loss: 2.4863 - accuracy: 0.0665 - val_loss: 2.4757 - val_accuracy: 0.1111
Epoch 2/50
21/21 [==============================] - 79s 3s/step - loss: 2.4756 - accuracy: 0.0988 - val_loss: 2.4670 - val_accuracy: 0.1333
Epoch 3/50
21/21 [==============================] - 75s 3s/step - loss: 2.4718 - accuracy: 0.1125 - val_loss: 2.4716 - val_accuracy: 0.1389
Epoch 4/50
21/21 [==============================] - 78s 4s/step - loss: 2.4718 - accuracy: 0.1174 - val_loss: 2.4638 - val_accuracy: 0.1333
Epoch 5/50
21/21 [==============================] - 78s 4s/step - loss: 2.4574 - accuracy: 0.1272 - val_loss: 2.4565 - val_accuracy: 0.1389
Epoch 6/50
21/21 [==============================] - 79s 3s/step - loss: 2.4556 - accuracy: 0.1164 - val_loss

In [15]:
# Add another hidden layer to fully connected part with dropout, increase learning rate
fig_accuracy

In [ ]:
train_data, valid_data, test_data = create_data(data_dir_name=data_dir_name, batch_size=50, pad_shape=pad_shape)

fig_loss = go.Figure(layout={'title': 'Loss'})
fig_accuracy = go.Figure(layout={'title': 'Accuracy'})

model = set_model(2.6e-3, 60, 0.4, pad_shape=pad_shape)

his = fit_model(model, train_data, valid_data, n_epochs=50)

fig_loss, fig_accuracy = plot_history(
    history=his,
    params=(2.6e-3, 60, 0.4),
    fig_loss=fig_loss,
    fig_accuracy=fig_accuracy,
)

metrics = evaluate_model(model, [train_data, valid_data, test_data])

Found 1202 files belonging to 12 classes.
Using 1022 files for training.
Found 1202 files belonging to 12 classes.
Using 180 files for validation.
Found 523 files belonging to 12 classes.
Epoch 1/50
21/21 [==============================] - 109s 4s/step - loss: 2.7182 - accuracy: 0.0939 - val_loss: 2.4583 - val_accuracy: 0.1333
Epoch 2/50
21/21 [==============================] - 104s 5s/step - loss: 2.6485 - accuracy: 0.1125 - val_loss: 2.4595 - val_accuracy: 0.1222
Epoch 3/50
21/21 [==============================] - 87s 4s/step - loss: 2.5537 - accuracy: 0.1135 - val_loss: 2.4643 - val_accuracy: 0.0944
Epoch 4/50
21/21 [==============================] - 96s 4s/step - loss: 2.5000 - accuracy: 0.1468 - val_loss: 2.4866 - val_accuracy: 0.0833
Epoch 5/50
21/21 [==============================] - 99s 4s/step - loss: 2.4233 - accuracy: 0.1634 - val_loss: 2.4746 - val_accuracy: 0.1167
Epoch 6/50
21/21 [==============================] - 111s 5s/step - loss: 2.3761 - accuracy: 0.1791 - val_loss:

In [20]:
# Add another hidden layer to fully connected part with dropout, add BatchNorm after first Dense before Dropout
fig_accuracy

In [ ]:
train_data, valid_data, test_data = create_data(data_dir_name=data_dir_name, batch_size=50, pad_shape=pad_shape)

fig_loss = go.Figure(layout={'title': 'Loss'})
fig_accuracy = go.Figure(layout={'title': 'Accuracy'})

model = set_model(2.6e-3, 60, 0.4, pad_shape=pad_shape)

his = fit_model(model, train_data, valid_data, n_epochs=50)

fig_loss, fig_accuracy = plot_history(
    history=his,
    params=(2.6e-3, 60, 0.4),
    fig_loss=fig_loss,
    fig_accuracy=fig_accuracy,
)

metrics = evaluate_model(model, [train_data, valid_data, test_data])

Found 1202 files belonging to 12 classes.
Using 1022 files for training.
Found 1202 files belonging to 12 classes.
Using 180 files for validation.
Found 523 files belonging to 12 classes.
Epoch 1/50
21/21 [==============================] - 106s 4s/step - loss: 2.8468 - accuracy: 0.1076 - val_loss: 2.4784 - val_accuracy: 0.1056
Epoch 2/50
21/21 [==============================] - 95s 4s/step - loss: 2.2339 - accuracy: 0.2309 - val_loss: 2.4760 - val_accuracy: 0.1389
Epoch 3/50
21/21 [==============================] - 100s 4s/step - loss: 2.0137 - accuracy: 0.3249 - val_loss: 2.4621 - val_accuracy: 0.1722
Epoch 4/50
21/21 [==============================] - 180s 9s/step - loss: 1.6805 - accuracy: 0.4697 - val_loss: 2.4484 - val_accuracy: 0.2222
Epoch 5/50
21/21 [==============================] - 136s 5s/step - loss: 1.3739 - accuracy: 0.5548 - val_loss: 2.4382 - val_accuracy: 0.1667
Epoch 6/50
21/21 [==============================] - 105s 4s/step - loss: 1.1202 - accuracy: 0.6243 - val_los

In [16]:
# Add BatchNorm after last conv2d and its maxpool
fig_accuracy

In [ ]:
train_data, valid_data, test_data = create_data(data_dir_name=data_dir_name, batch_size=50, pad_shape=pad_shape)

fig_loss = go.Figure(layout={'title': 'Loss'})
fig_accuracy = go.Figure(layout={'title': 'Accuracy'})

model = set_model(2.6e-3, 60, 0.4, pad_shape=pad_shape)

his = fit_model(model, train_data, valid_data, n_epochs=50)

fig_loss, fig_accuracy = plot_history(
    history=his,
    params=(2.6e-3, 60, 0.4),
    fig_loss=fig_loss,
    fig_accuracy=fig_accuracy,
)

metrics = evaluate_model(model, [train_data, valid_data, test_data])

Found 1202 files belonging to 12 classes.
Using 1022 files for training.
Found 1202 files belonging to 12 classes.
Using 180 files for validation.
Found 523 files belonging to 12 classes.
Epoch 1/50
21/21 [==============================] - 503s 24s/step - loss: 7.2918 - accuracy: 0.1076 - val_loss: 2.7369 - val_accuracy: 0.1167
Epoch 2/50
21/21 [==============================] - 339s 16s/step - loss: 2.5931 - accuracy: 0.1468 - val_loss: 3.7115 - val_accuracy: 0.1167
Epoch 3/50
21/21 [==============================] - 408s 19s/step - loss: 2.4278 - accuracy: 0.1566 - val_loss: 4.4159 - val_accuracy: 0.1278
Epoch 4/50
21/21 [==============================] - 261s 11s/step - loss: 2.4279 - accuracy: 0.1566 - val_loss: 5.8639 - val_accuracy: 0.1333
Epoch 5/50
21/21 [==============================] - 110s 5s/step - loss: 2.4120 - accuracy: 0.1575 - val_loss: 6.3387 - val_accuracy: 0.1278
Epoch 6/50
21/21 [==============================] - 475s 23s/step - loss: 2.3829 - accuracy: 0.1605 - v

In [19]:
# Add BatchNorm after 2nd and 3d conv2d after maxpool
fig_accuracy

# Hyperparameters optimization

In [20]:
# Hyperparameters grid

params_grid = dict(
    lr=np.arange(1e-3, 4e-3, 2e-4),
    n=np.arange(60, 70, 5),
    p=np.arange(0.3, 0.6, 0.1),
)

count = 1
for value in params_grid.values():
    count *= len(value)
count

90

In [ ]:
fig_loss = go.Figure(layout={'title': 'Loss'})
fig_accuracy = go.Figure(layout={'title': 'Accuracy'})

n_epochs=20

for i, params in enumerate(product(params_grid['lr'], params_grid['n'], params_grid['p'])):
    print(f'START: {i}, {params}')

    train_data, valid_data, test_data = create_data(data_dir_name=data_dir_name, batch_size=50, pad_shape=pad_shape)

    lr, n, p = params
    model = set_model(lr, n, p, pad_shape=pad_shape)

    his = fit_model(model, train_data, valid_data, n_epochs)

    fig_loss, fig_accuracy = plot_history(
        history=his,
        params=params,
        fig_loss=fig_loss,
        fig_accuracy=fig_accuracy,
    )

    fig_loss.write_html('hyperparams_loss.html')
    fig_accuracy.write_html('hyperparams_accuracy.html')

# Build model with best hyperparameters

In [9]:
fig_loss = go.Figure(layout={'title': 'Loss'})
fig_accuracy = go.Figure(layout={'title': 'Accuracy'})

n_epochs = 16

train_data, valid_data, test_data = create_data(data_dir_name=data_dir_name, batch_size=50, pad_shape=pad_shape)

lr, n, p = np.arange(1e-3, 4e-3, 2e-4)[2], 60, 0.4 # Best hyperparams
model = set_model(lr, n, p, pad_shape=pad_shape)

his = fit_model(model, train_data, valid_data, n_epochs)

fig_loss, fig_accuracy = plot_history(
    history=his,
    params=(lr, n, p),
    fig_loss=fig_loss,
    fig_accuracy=fig_accuracy,
)

fig_loss.write_html('model_best_loss.html')
fig_accuracy.write_html('nodel_best_accuracy.html')

Found 1202 files belonging to 12 classes.
Using 1022 files for training.
Found 1202 files belonging to 12 classes.
Using 180 files for validation.
Found 523 files belonging to 12 classes.


Epoch 1/16


21/21 [==============================] - 94s 4s/step - loss: 2.9903 - accuracy: 0.1164 - val_loss: 2.4784 - val_accuracy: 0.1222
Epoch 2/16
21/21 [==============================] - 111s 5s/step - loss: 2.3113 - accuracy: 0.2045 - val_loss: 2.4675 - val_accuracy: 0.1167
Epoch 3/16
21/21 [==============================] - 135s 6s/step - loss: 2.0599 - accuracy: 0.3239 - val_loss: 2.4589 - val_accuracy: 0.2500
Epoch 4/16
21/21 [==============================] - 229s 11s/step - loss: 1.8371 - accuracy: 0.3816 - val_loss: 2.4446 - val_accuracy: 0.2667
Epoch 5/16
21/21 [==============================] - 300s 14s/step - loss: 1.6454 - accuracy: 0.4726 - val_loss: 2.4301 - val_accuracy: 0.2389
Epoch 6/16
21/21 [==============================] - 330s 16s/step - loss: 1.4492 - accuracy: 0.5215 - 

In [10]:
evaluate_model(model, [train_data, valid_data, test_data])

11/11 [==============================] - 10s 822ms/step - loss: 2.1801 - accuracy: 0.3595


[[1.3873101472854614, 0.9011741876602173],
 [2.1659295558929443, 0.4166666567325592],
 [2.180077075958252, 0.3594646155834198]]

In [11]:
model.save('model.keras')